In [65]:
!pip3 install google-cloud-speech google-cloud google-cloud-texttospeech langdetect

4111.49s - pydevd: Sending message related to process being replaced timed-out after 5 seconds


Defaulting to user installation because normal site-packages is not writeable
     |████████████████████████████████| 981 kB 1.6 MB/s eta 0:00:01
  Created wheel for langdetect: filename=langdetect-1.0.9-py3-none-any.whl size=993242 sha256=43b4b1e9511d1cfc04d4537a373d37f0dfe08d3c0852b45a8d2269cdbf511776
  Stored in directory: /Users/mnlee/Library/Caches/pip/wheels/d1/c1/d9/7e068de779d863bc8f8fc9467d85e25cfe47fa5051fff1a1bb
Successfully built langdetect
You should consider upgrading via the '/Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip' command.


In [21]:
BUCKET_NAME = "mnlee-stt" # @param {type:"string"}

In [31]:
def synthesize_text(lang_code, text, audio_name):
    """Synthesizes speech from the input string of text."""
    from google.cloud import texttospeech

    client = texttospeech.TextToSpeechClient()

    input_text = texttospeech.SynthesisInput(text=text)

    # Note: the voice can also be specified by name.
    # Names of voices can be retrieved with client.list_voices().
    voice = texttospeech.VoiceSelectionParams(
        language_code=lang_code,
        #name="en-US-Standard-C",
        ssml_gender=texttospeech.SsmlVoiceGender.FEMALE,
    )

    audio_config = texttospeech.AudioConfig(
        audio_encoding=texttospeech.AudioEncoding.LINEAR16
    )

    response = client.synthesize_speech(
        request={"input": input_text, "voice": voice, "audio_config": audio_config}
    )

    # The response's audio_content is binary.
    with open(f"{audio_name}.wav", "wb") as out:
        out.write(response.audio_content)
        print(f"Audio content written to file {audio_name}.wav")

In [3]:
synthesize_text("en-US","Hello. This is a sentence to test multilingual language detection.", "en-US")

/Users/mnlee/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


Audio content written to file en-US.wav


In [5]:
synthesize_text("es-ES","Hola. Esta es una oración para probar la detección de lenguaje multilingüe.", "es-ES")

Audio content written to file es-ES.wav


In [6]:
synthesize_text("fr-FR","Bonjour. Ceci est une phrase pour tester la détection de langue multilingue.", "fr-FR")

Audio content written to file fr-FR.wav


In [7]:
synthesize_text("de-DE","Hallo. Dies ist ein Satz, um die mehrsprachige Spracherkennung zu testen.", "de-DE")

Audio content written to file de-DE.wav


In [8]:
synthesize_text("pt-PT","Olá. Esta é uma frase para testar a detecção de idioma multilíngue.", "pt-PT")

Audio content written to file pt-PT.wav


In [9]:
synthesize_text("cmn-CN","你好。这是一句用于测试多语言语言检测的句子。", "cmn-CN")

Audio content written to file cmn-CN.wav


In [10]:
synthesize_text("ja-JP","こんにちは。これは多言語言語検出をテストするための文です。", "ja-JP")

Audio content written to file ja-JP.wav


In [11]:
synthesize_text("ko-KR","안녕하세요. 다국어 언어 감지를 테스트하기 위한 문장입니다.", "ko-KR")

Audio content written to file ko-KR.wav


In [41]:
synthesize_text("ko-KR","hello good morning. 영한 혼합 문장입니다", "ko-en-word")

Audio content written to file ko-en-word.wav


In [12]:
synthesize_text("ar-AE","مرحبًا. هذه جملة لاختبار اكتشاف اللغة متعددة اللغات.", "ar-AE")

Audio content written to file ar-AE.wav


In [60]:
synthesize_text("hi-IN","नमस्ते। यह बहुभाषी भाषा पहचान का परीक्षण करने के लिए एक वाक्य है.", "hi-IN")

Audio content written to file hi-IN.wav


In [ ]:
#!gsutil cp ko-en-word2.wav gs://mnlee-stt/robot/wav/
!gsutil cp *.wav gs://mnlee-stt/robot/wav/

In [72]:
from google.cloud import speech_v1p1beta1 as speech
from langdetect import detect
#from google.cloud import speech_v2 as speech

#def stt(wav_uri) -> speech.RecognizeResponse:
def stt(lang_code):
    # Instantiates a client
    client = speech.SpeechClient()
    audio_uri = f"gs://{BUCKET_NAME}/robot/wav/{lang_code}.wav"
    audio = speech.RecognitionAudio(uri=audio_uri)

    config = speech.RecognitionConfig(
        encoding=speech.RecognitionConfig.AudioEncoding.LINEAR16,
        language_code="en-US",
        sample_rate_hertz=24000,
        #sample_rate_hertz=48000,
        #alternative_language_codes=["es-ES","fr-FR","de-DE","pt-PT","cmn-CN","ja-JP","ko-KR", "ar-AE", "hi-IN"]
        alternative_language_codes=["es-ES","fr-FR","de-DE","pt-PT","cmn-CN","ko-KR"]
        maxAlternatives=2,
        enableSpokenPunctuation=True
    )

    # Detects speech in the audio file
    response = client.recognize(config=config, audio=audio)
    #print(response)
    for result in response.results:
        for alternative in result.alternatives:
            print("Transcript: {}".format(result.alternatives.transcript))    
            print("Detected [lang code]]: {}".format(detect(result.alternatives.transcript)))
        print("STT [lang code]: {}".format(result.language_code))
          
    #return response

In [78]:
stt("en-US")

Transcript: hello this is a descendants 2 test multilingual language detection
lang code: en-us


In [79]:
stt("es-ES")

Transcript: hola esta es una oración para aprobar la detección de lenguaje multilingüe
lang code: es-es


In [80]:
stt("fr-FR")

Transcript: bonjour ceci est une phrase pour tester la détection de langue multilingue
lang code: fr-fr


In [81]:
stt("de-DE")

Transcript: hallo dies ist ein Satz um die mehrsprachige Spracherkennung zu testen
lang code: de-de


In [82]:
stt("pt-PT")

Transcript: Olá esta é uma frase para testar a detecção de idioma multilingue
lang code: pt-pt


In [83]:
stt("cmn-CN")

Transcript: 你好这是一句用于测试多语言语言检测的句子
lang code: cmn-hans-cn


In [84]:
stt("ko-KR")

Transcript: 안녕하세요 다국어 언어 감지를 테스트 하기 위한 문장입니다
lang code: ko-kr


In [ ]:
stt("ja-JP")

In [ ]:
stt("ar-AE")

In [ ]:
stt("hi-IN")

In [68]:
stt("en-ko-word2")

Transcript: my favorite food is 김치찌개
STT [lang code]: ko-kr
Detected [lang code]]: en


In [70]:
stt("ko-en-word2")

Transcript: 김치찌개 is my favorite food
STT [lang code]: ko-kr
Detected [lang code]]: en


## audio concat

In [22]:
!pip3 install pydub

892.87s - pydevd: Sending message related to process being replaced timed-out after 5 seconds


Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip' command.


In [77]:
from pydub import AudioSegment

en = AudioSegment.from_wav("en-US.wav")
ko = AudioSegment.from_wav("ko-KR.wav")
es = AudioSegment.from_wav("es-ES.wav")
fr = AudioSegment.from_wav("fr-FR.wav")
silence = AudioSegment.silent(duration=2000)
en_ko = en + silence + ko
en_ko.export("en_ko.wav", format="wav")
es_fr = es + silence + fr
es_fr.export("es_fr.wav", format="wav")
en_ko_es_fr = en + silence + ko + silence + es + silence + fr
en_ko_es_fr.export("en_ko_es_fr.wav", format="wav")

<_io.BufferedRandom name='en_ko_es_fr.wav'>

In [78]:
stt("en_ko")

Transcript: hello this is a sentence to test multilingual language detection
STT [lang code]: en-us
Detected [lang code]]: en
Transcript:  I know that's how I will go and I'll give you
STT [lang code]: en-us
Detected [lang code]]: en


In [79]:
stt("es_fr")

Transcript: tu n'as pas de la détection de bonjour ceci est une phrase pour tester la détection de langue multilingue
STT [lang code]: fr-fr
Detected [lang code]]: fr


In [80]:
stt("en_ko_es_fr")

Transcript: hello this is a sentence to test multilingual language detection
STT [lang code]: en-us
Detected [lang code]]: en
Transcript:  I know that's how I will go and I'll give you
STT [lang code]: en-us
Detected [lang code]]: en
Transcript:  hola senora multi-language
STT [lang code]: en-us
Detected [lang code]]: ca
Transcript:  Google search in Facebook they still addiction to Long Beach Island
STT [lang code]: en-us
Detected [lang code]]: en
